In [3]:
# tools/get_bbox.py
import geopandas as gpd
from pathlib import Path
import atlite

fp = Path("./bus_voronoi.gpkg")  # <- update if different
gdf = gpd.read_file(fp).to_crs(4326)
minx, miny, maxx, maxy = gdf.total_bounds
print(f"x: [{minx:.4f}, {maxx:.4f}]  y: [{miny:.4f}, {maxy:.4f}]")

x: [-10.4782, 31.5695]  y: [36.0065, 71.1271]


In [4]:
import atlite
import geopandas as gpd
import shapely
from shapely.geometry import box
from atlite.resource import get_windturbineconfig

# ---------- INPUTS ----------
CUTOUT_PATH = "europe-2012-sarah3-era5.nc"   # downloaded from Zenodo
VORONOI_PATH = "./bus_voronoi.gpkg"          # your Voronoi file
ID_COL = "bus_id"                            # set to None if no ID column
HUB_HEIGHT_M = 100                           # wind hub height
# ----------------------------

# 0) Load inputs
cutout = atlite.Cutout(CUTOUT_PATH)
gdf = gpd.read_file(VORONOI_PATH).to_crs(4326)

# 1) Build cutout bounding box (in 4326), then project to 3035
xmin = float(cutout.data.x.min()); xmax = float(cutout.data.x.max())
ymin = float(cutout.data.y.min()); ymax = float(cutout.data.y.max())
bbox_4326 = box(xmin, ymin, xmax, ymax)
bbox_3035 = gpd.GeoSeries([bbox_4326], crs=4326).to_crs(3035).iloc[0]

# 2) Clean polygons in projected CRS (ETRS89 / LAEA Europe = EPSG:3035)
gdf_proj = gdf.to_crs(3035)

# make valid (Shapely ≥2 has validation.make_valid; fallback: buffer(0))
try:
    from shapely.validation import make_valid as _make_valid
    gdf_proj["geometry"] = gdf_proj.geometry.apply(_make_valid)
except Exception:
    gdf_proj["geometry"] = gdf_proj.geometry.buffer(0)

# snap to a 1 m precision grid (kills micro self-intersections)
gdf_proj["geometry"] = gdf_proj.geometry.apply(lambda geom: shapely.set_precision(geom, 1.0))

# drop empties/invalids and tiny slivers (< 1 m²), gentle snap for spikes
gdf_proj = gdf_proj[~gdf_proj.geometry.is_empty & gdf_proj.geometry.is_valid].copy()
gdf_proj["area_m2"] = gdf_proj.geometry.area
gdf_proj = gdf_proj[gdf_proj["area_m2"] > 1.0].drop(columns="area_m2")
gdf_proj["geometry"] = gdf_proj.geometry.buffer(0.01).buffer(-0.01)  # ~1 cm

# clip *in projected CRS* and final clean
gdf_proj["geometry"] = gdf_proj.geometry.intersection(bbox_3035)
gdf_proj["geometry"] = gdf_proj.geometry.buffer(0)
gdf_proj = gdf_proj[~gdf_proj.geometry.is_empty & gdf_proj.geometry.is_valid].copy()

# back to WGS84 for atlite
gdf = gdf_proj.to_crs(4326)

# explode/dissolve per ID if needed
if ID_COL and ID_COL in gdf.columns:
    gdf = gdf.explode(index_parts=False).dropna(subset=["geometry"])
    gdf = gdf.dissolve(by=ID_COL, as_index=False)
    labels = gdf[ID_COL].astype(str)
else:
    gdf = gdf.explode(index_parts=False).dropna(subset=["geometry"])
    labels = gdf.index.astype(str)

# shapes GeoSeries with stable labels (used as output columns)
shapes = gpd.GeoSeries(gdf.geometry.values, index=labels)

# 3) WIND capacity-factor time series aggregated to polygons (atlite 0.4.1)
turb = get_windturbineconfig("Vestas_V112_3MW")
turb["hub_height"] = HUB_HEIGHT_M

cf_wind = cutout.wind(
    turbine=turb,
    shapes=shapes, shapes_crs=4326,
    per_unit=True,
    capacity_factor_timeseries=True,   # keep the time dimension
    add_cutout_windspeed=True,
    interpolation_method="logarithmic",
)
# (optional) rechunk by time if available
if "time" in cf_wind.dims:
    cf_wind = cf_wind.chunk({"time": 240})

cf_wind.to_pandas().to_csv("cf_wind_by_cell_2012.csv")

# 4) PV capacity-factor time series aggregated to polygons
cf_pv = cutout.pv(
    panel="CSi",
    orientation="latitude_optimal",    # or {"tilt": 35, "azimuth": 180}
    tracking=None,                     # None = no tracking (0.4.x API)
    shapes=shapes, shapes_crs=4326,
    per_unit=True,
    capacity_factor_timeseries=True,
)
if "time" in cf_pv.dims:
    cf_pv = cf_pv.chunk({"time": 240})

cf_pv.to_pandas().to_csv("cf_pv_by_cell_2012.csv")

print("Wrote: cf_wind_by_cell_2010.csv, cf_pv_by_cell_2012.csv")


C:\Users\au710726\Anaconda3\envs\pypsa-eur2\Lib\site-packages\atlite\resource.py:90: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.15.
  warnings.warn(msg, FutureWarning)


Wrote: cf_wind_by_cell_2010.csv, cf_pv_by_cell_2012.csv


In [ ]:
from atlite.resource import get_windturbineconfig

# WIND
turb = get_windturbineconfig("Vestas_V112_3MW"); turb["hub_height"] = 100
cf_wind = cutout.wind(
    turbine=turb,
    shapes=shapes, shapes_crs=4326,
    per_unit=True,
    capacity_factor_timeseries=True,
    add_cutout_windspeed=True,
    interpolation_method="logarithmic",
)
cf_wind.to_pandas().to_csv("cf_wind_by_cell_2010.csv")

# PV  (note tracking=None for no tracking in 0.4.1)
cf_pv = cutout.pv(
    panel="CSi",
    orientation="latitude_optimal",
    tracking=None,
    shapes=shapes, shapes_crs=4326,
    per_unit=True,
    capacity_factor_timeseries=True,
)
cf_pv.to_pandas().to_csv("cf_pv_by_cell_2010.csv")


C:\Users\au710726\Anaconda3\envs\pypsa-eur2\Lib\site-packages\atlite\resource.py:90: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.15.
  warnings.warn(msg, FutureWarning)
